#### 개인과제 !!!!!! ㅇ0ㅇ

##### 주피터랩에서 데이터를 입력받으면 3가지 DB에 동시에 CRUD가 되는 시스템을 만들어라 !!! 

ex) input을 이용하여 <이름> : 홍길동 (엔터), <나이> 33 (엔터), <주소> 해운대구 (엔터)
입력하고, 데이터를 검색하고, 이름을 고르고 수정하고, 삭제 가능하게 하라 !

In [14]:
##예제 !!
str = """
    1. 데이터 등록
    2. 데이터 읽기
    3. 데이터 수정
    4. 데이터 삭제
    5. 종료
    고르세요.
    """
while True:
    print(str)
    inp = input("선택하세요 ! : ")
    print(inp,'고르셨군요 !')
    if inp =='1':
        inp1= input("이름을 입력하세요 !")
        inp2= input("나이를 입력하세요 !")
        inp3= input("주소를 입력하세요 !")
    if inp =='2' or '5':
        print("종료하겠습니다.")
        break


    1. 데이터 등록
    2. 데이터 읽기
    3. 데이터 수정
    4. 데이터 삭제
    5. 종료
    고르세요.
    


선택하세요 ! :  1


1 고르셨군요 !


이름을 입력하세요 ! hong
나이를 입력하세요 ! 20
주소를 입력하세요 ! busan


종료하겠습니다.


In [2]:
import mysql.connector
import oracledb
from pymongo import MongoClient

# MySQL 연결
try:
    mysql_conn = mysql.connector.connect(
        host="mysql",
        user="root",
        password="1234"
    )
    if mysql_conn.is_connected():
        print("MySQL 서버에 성공적으로 연결되었습니다.")
        mysql_cursor = mysql_conn.cursor()
        mysql_cursor.execute("CREATE DATABASE IF NOT EXISTS testdb")
        mysql_conn.close()

    mysql_conn = mysql.connector.connect(
        host="mysql",
        user="root",
        password="1234",
        database="testdb"
    )
    mysql_cursor = mysql_conn.cursor()
    print("MySQL 연결 성공!")
except Exception as err:
    print("MySQL 연결 실패!", err)

# Oracle 연결
try:
    oracle_conn = oracledb.connect(
        user="pknu",
        password="1234",
        dsn="oracle/XEPDB1"
    )
    print("Oracle 데이터베이스에 성공적으로 연결 되었습니다!")
    oracle_cursor = oracle_conn.cursor()
except Exception as e:
    print("Oracle 연결 실패:", e)

# MongoDB 연결
client = MongoClient("mongodb://mongodb:27017")
try:
    client.admin.command("ping")
    print("MongoDB 서버에 성공적으로 연결되었습니다.")
except Exception as e:
    print("MongoDB 연결 실패:", e)

# MongoDB DB & Collection 선택
db = client["tdb"]
collection = db["t1"]

# Oracle 테이블 생성
oracle_cursor.execute("""
BEGIN
    EXECUTE IMMEDIATE '
        CREATE TABLE test (
            id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
            name VARCHAR2(50),
            age NUMBER,
            address VARCHAR2(100)
        )';
EXCEPTION
    WHEN OTHERS THEN
        IF SQLCODE != -955 THEN
            RAISE;
        END IF;
END;
""")
print("Oracle 테이블 생성 완료!")

# MySQL 테이블 생성
mysql_cursor.execute("""
CREATE TABLE IF NOT EXISTS test (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    age INT,
    address VARCHAR(100)
)
""")
print("MySQL 테이블 생성 완료!")

MySQL 서버에 성공적으로 연결되었습니다.
MySQL 연결 성공!
Oracle 데이터베이스에 성공적으로 연결 되었습니다!
MongoDB 서버에 성공적으로 연결되었습니다.
Oracle 테이블 생성 완료!
MySQL 테이블 생성 완료!


In [3]:
str = """
    1. 데이터 등록
    2. 데이터 읽기
    3. 데이터 수정
    4. 데이터 삭제
    5. 종료
    고르세요.
"""
while True:
    print(str)
    inp = input("선택하세요 ! : ")
    print(inp, '번을 고르셨군요 !')

    if inp == '1':
        name1 = input("저장하실 이름을 입력해 주세요 ! : ")
        age1 = input("나이를 입력해 주세요 ! : ")
        address1 = input("주소를 입력해 주세요 ! : ")

        # MySQL Create
        sql_mysql = "INSERT INTO test (name, age, address) VALUES (%s, %s, %s)"
        mysql_cursor.execute(sql_mysql, (name1, age1, address1))
        mysql_conn.commit()

        # Oracle Create
        sql_oracle = "INSERT INTO test (name, age, address) VALUES (:1, :2, :3)"
        oracle_cursor.execute(sql_oracle, (name1, age1, address1))
        oracle_conn.commit()

        # MongoDB Create
        test_doc = {"name": name1, "age": age1, "address": address1}
        collection.insert_one(test_doc)

        print("모든 DB에 데이터를 등록 완료했습니다! ^_^")

    elif inp == '2':
        name2 = input("조회하실 이름을 입력하세요 ! :")

        # MySQL Select
        mysql_cursor.execute("SELECT * FROM test WHERE name = %s", (name2,))
        results = mysql_cursor.fetchall()
        for f in results:
            print(f"{f} : MySQL에서 조회된 정보입니다.")

        # Oracle Select
        oracle_cursor.execute("SELECT * FROM test WHERE name = :1", (name2,))
        oracle_results = oracle_cursor.fetchall()
        for r in oracle_results:
            print(f"{r} : Oracle에서 조회된 정보입니다.")

        # MongoDB Select
        mongo_result = collection.find_one({"name": name2}, {"_id": 0})
        print(f"{mongo_result} : MongoDB에서 조회된 정보입니다.")

    elif inp == '3':
        name3 = input("수정하실 대상의 이름을 입력하세요 !: ")
        what3 = input("수정하실 항목을 입력하세요 ! (예: age, address): ")
        about3 = input("수정하실 내용: ")

        # MySQL Update
        sql = f"UPDATE test SET {what3} = %s WHERE name = %s"
        mysql_cursor.execute(sql, (about3, name3))
        mysql_conn.commit()

        # Oracle Update
        sql_oracle = f"UPDATE test SET {what3} = :1 WHERE name = :2"
        oracle_cursor.execute(sql_oracle, (about3, name3))
        oracle_conn.commit()

        # MongoDB Update
        collection.update_one({"name": name3}, {"$set": {what3: about3}})

        print("모든 DB에서 수정 완료했습니다! ^_^")

    elif inp == '4':
        name4 = input("삭제하실 대상의 이름을 입력하세요 !")

        # MySQL Delete
        mysql_cursor.execute("DELETE FROM test WHERE name = %s", (name4,))
        mysql_conn.commit()

        # Oracle Delete
        oracle_cursor.execute("DELETE FROM test WHERE name = :1", (name4,))
        oracle_conn.commit()

        # MongoDB Delete
        collection.delete_one({"name": name4})

        print("모든 DB에서 삭제 완료했습니다! ^_^")

    elif inp == '5':
        print("종료하겠습니다.")
        break


    1. 데이터 등록
    2. 데이터 읽기
    3. 데이터 수정
    4. 데이터 삭제
    5. 종료
    고르세요.



선택하세요 ! :  1


1 고르셨군요 !


이름을 입력하세요 ! :  용감한쿠키
나이를 입력하세요 ! :  1
주소를 입력하세요 ! :  쿠킹덤


모든 DB에 데이터가 등록되었습니다!

    1. 데이터 등록
    2. 데이터 읽기
    3. 데이터 수정
    4. 데이터 삭제
    5. 종료
    고르세요.



...

3 고르셨군요 !


수정하실 대상의 이름을 입력하세요 !:  용감한쿠키
수정하실 항목을 입력하세요 ! (예: age, address):  age
수정하실 내용:  9999


모든 DB에서 수정 완료되었습니다.

    1. 데이터 등록
    2. 데이터 읽기
    3. 데이터 수정
    4. 데이터 삭제
    5. 종료
    고르세요.



선택하세요 ! :  5


5 고르셨군요 !
종료하겠습니다.
